
# 📝 **Práctica #2:** 🚀  Entrenamiento de una red neuronal para la clasificación de caras de actores.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LoboaTeresa/Workshop-COF-23/blob/master/Practica2__VGG_X_Ray.ipynb)

# **Práctica #2.1.:** Entrenamiento desde cero

### 1. Importar librerías necesarias

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

### 2. Cargar el dataset

Trabajaremos con un dataset con imágenes de caras de 17 actores y actrices. El dataset contiene 1000 imágenes de cada actor.


In [ ]:
# descarga el dataset:
!git clone https://github.com/LoboaTeresa/Workshop-COF-23.git
!cp /content/Workshop-COF-23/assets/Celebrity_Faces_Dataset.zip /content/Celebrity_Faces_Dataset.zip
!unzip /content/Celebrity_Faces_Dataset.zip

In [3]:
train_path="Celebrity_Faces_Dataset/train"
test_path="Celebrity_Faces_Dataset/test"
val_path="Celebrity_Faces_Dataset/val"
data_path = [train_path, test_path, val_path]

In [11]:
x_train = []
x_test = []
x_val = []

x_data = [x_train, x_test, x_val]

VGG_INPUT_SIZE = (160,160,3)
image_size = VGG_INPUT_SIZE[:-1]

for i, subset in enumerate(x_data):
    for folder in os.listdir(data_path[i]):
        sub_path = data_path[i] + "/" + folder

        for img in os.listdir(sub_path):
            image_path = sub_path + "/" + img
            img_arr = cv2.imread(image_path)
            img_arr = cv2.resize(img_arr,image_size)
            subset.append(img_arr)


In [12]:
# Normalizar imágenes para obtener mayor rendimiento
train_x = np.array(x_train)
test_x = np.array(x_test)
val_x = np.array(x_val)
train_x = train_x / 255.0
test_x = test_x / 255.0
val_x = val_x / 255.0

Obtendremos las anotaciones a partir de la estructura de carpetas de nuestro dataset usando ImageDataGenerator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = image_size,
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = image_size,
                                            batch_size = 32,
                                            class_mode = 'categorical')
val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = image_size,
                                            batch_size = 32,
                                            class_mode = 'categorical')

n_classes = list(training_set.class_indices.keys())
class_names = len(n_classes)

# labels
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes

### 3. Definición de la arquitectura:

Keras es una librería de código abierto que proporciona una interfaz Python para redes neuronales artificiales. Keras actúa como interfaz para la librería TensorFlow desarrollada por Google.

[Documentación de Keras](https://keras.io/api/layers/). Échale un ojo :)

Vamos a construir una red neuronal de 4 capas densas o completamente conectadas.

![VGG16](https://www.researchgate.net/publication/350828239/figure/fig1/AS:1017585780924416@1619622764106/Architecture-of-the-modified-VGG16-model.ppm)

* 2 x convolution layer of 64 channel of 3x3 kernel and same padding.
* 1 x maxpool layer of 2x2 pool size and stride 2x2.
* 2 x convolution layer of 128 channel of 3x3 kernel and same padding.
* 1 x maxpool layer of 2x2 pool size and stride 2x2.
* 3 x convolution layer of 256 channel of 3x3 kernel and same padding.
* 1 x maxpool layer of 2x2 pool size and stride 2x2.
* 3 x convolution layer of 512 channel of 3x3 kernel and same padding.
* 1 x maxpool layer of 2x2 pool size and stride 2x2.
* 3 x convolution layer of 512 channel of 3x3 kernel and same padding.
* 1 x maxpool layer of 2x2 pool size and stride 2x2.
* 1 x Dense layer of 4096 units.
* 1 x Dense layer of 4096 units.
* 1 x Dense Softmax layer of 18 units (there are 18 classes).

We also added the rectified linear unit (ReLU) activation to each layer so that the negative values aren’t passed to the next layer. The last layer will have a softmax activation function.

In [20]:
class_names = list(training_set.class_indices.keys())
n_classes = len(class_names)

In [15]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

model = Sequential()  #  initializing the model

model.add(Conv2D(input_shape=VGG_INPUT_SIZE,filters=64,kernel_size=(3,3),padding="same", activation="relu"))  # tamaño de las imágenes: 224, 224, 3 colores
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))  # 4096 neuronas
model.add(Dense(units=4096,activation="relu"))  # 4096 neuronas
model.add(Dense(units=n_classes, activation="softmax"))  # 17 neuronas, 17 actores

### 4. Compilar el modelo

In [ ]:
model.compile(optimizer="adam", loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
model.summary()

### 5. Entrenar el modelo

In [ ]:
history = model.fit(x=training_set, validation_data=val_set, epochs=2, steps_per_epoch=2, validation_steps=2)

### 7. Inferencia: hora de ponerlo a prueba!

In [ ]:
from keras.preprocessing import image

img = image.load_img("/content/Celebrity_Faces_Dataset/test/Denzel_Washington/094_7858a9ff.jpg",target_size=image_size)
img = np.asarray(img)
plt.imshow(img)
img = np.expand_dims(img, axis=0)
output = model.predict(img)
print(class_names[np.argmax(output)])

# **Práctica #2.2.:** Entrenamiento a partir de un modelo pre-entrenado

### 1. Cargar el modelo pre entrenado

In [31]:
from keras.applications.vgg16 import VGG16

vgg_model = Sequential()

pretrained_model= keras.applications.VGG16(include_top=False,
                   input_shape=VGG_INPUT_SIZE,
                   pooling='avg',classes=?,
                   weights='imagenet')

# Congelar las capas del modelo pre-entrenado: especificar que los pesos no cambien  durante el entrenamiento
for layer in pretrained_model.layers:
        layer.trainable=False

# Añadir un par de capas extras, la última con 17 neuronas (n_classes)
vgg_model.add(pretrained_model)
vgg_model.add(_INSERT_LAYER_HERE_())
vgg_model.add(_INSERT_LAYER_HERE_(512, activation='relu'))
vgg_model.add(_INSERT_LAYER_HERE_(?, activation='softmax'))  # Capa de predicción. SOftmax se suele usar para tareas de clasificaciónmulti-categoría.

### 2. Compilar el modelo

In [ ]:
vgg_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
vgg_model.summary()

### 3. Entrenar modelo

In [ ]:
history = vgg_model.fit(training_set, epochs=2, batch_size = 32)

In [ ]:
losss = history.history['loss']
accu = history.history['accuracy']

f, axarr = plt.subplots(1,2, figsize=(10,8))
axarr[0].plot(losss)
axarr[0].set_title("Loss")
axarr[0].set_xlabel('Epochs')
axarr[1].plot(accu)
axarr[1].set_title("Accuracy")
axarr[1].set_xlabel('Epochs')


### 4. Evaluar Modelo

Matriz de confusión: es una herramienta que permite visualizar el desempeño de un algoritmo  de aprendizaje supervisado. Cada columna de la matriz representa el número de predicciones de cada clase, mientras que cada fila representa a las instancias en la clase real., o sea en términos prácticos nos permite ver  qué tipos de aciertos y errores está teniendo nuestro modelo a la hora de pasar por el proceso de aprendizaje con los datos.

<img src="https://github.com/LoboaTeresa/Workshop-COF-23/blob/main/assets/CM_explained.png?raw=true"
     alt="confusionMatrix"
     width=500 />

In [ ]:
import sklearn.metrics as metrics

outputs = vgg_model.predict(test_set)
y_pred = np.argmax(outputs, axis=1)
cm = metrics.confusion_matrix(test_y, y_pred)
class_ids = list(range(len(class_names)))
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = class_ids)
cm_display.plot()
plt.figure(figsize=(50, 50), dpi=80)
plt.show()

In [ ]:
# relación nombre-índice:
training_set.class_indices

### 5. Hacer predicciones (inferencia). ¡Hora de ponerlo a prueba!

In [ ]:
img = image.load_img("/content/Celebrity_Faces_Dataset/test/Denzel_Washington/094_7858a9ff.jpg",target_size=image_size)
img = np.asarray(img)
plt.imshow(img)
img = np.expand_dims(img, axis=0)
output = vgg_model.predict(img)
print(class_names[np.argmax(output)])